# Masking Policies & Clones

IHAC ask whether or not masking policies were copied when a database is cloned. This demonstration shows that those policies are abosolutely cloned.

## Create Initial Objects

Make an environment to run our 

In [ ]:
-- ################################# Create Initial Objects
use role accountadmin;

-- create warehouse
create or replace warehouse demo_wh
    with
        warehouse_size = 'XSmall'
        auto_suspend = 30;
        
-- issue grants to public role - we will use this to test policies
grant usage on warehouse demo_wh to public;

create or replace database mp_test;
create or replace schema mp_test.test;
create or replace table mp_test.test.customer as 
(
    select
        *
    from
        snowflake_sample_data.tpch_sf1.customer 
    limit 100
);

-- grant usage to mp_test
grant usage on database mp_test to role public;
grant usage on schema mp_test.test to role public;
grant select on table mp_test.test.customer to role public;

select 'Objects created successfully!' as results;

### Preview the Data
Let's preview the data using the role `ACCOUNTADMIN`

In [ ]:
use role accountadmin;
select
    *
from
    mp_test.test.customer;

### Compare the Preview
Now let's look at the data using the `PUBLIC` role. You'll notice, the results are identical.

In [ ]:
-- preview data as public w/o a masking policy
use role public;
select
    *
from
    mp_test.test.customer;

## Create Masking Policies

A simple masking policies which will obscure phone numbers unless you're using the `ACCOUNDADMIN` role.

In [ ]:
use role accountadmin;
-- create masking policy
create or replace masking policy mp_test.public.phone_mask as (val string) returns string ->
    case
        when current_role() in ('ACCOUNTADMIN') then val
        else '**-***-***-***'
    end;

-- apply masking policy
alter table mp_test.test.customer
    alter column c_phone
        set masking policy mp_test.public.phone_mask;

### Let's Look at the data again
Using the `ACCOUNTADMIN` role, we can see the same results as above.

In [ ]:
-- preview data
use role accountadmin;
select
    *
from
    mp_test.test.customer;

### Masking policy at work
However, now we can see the masking policy at work when querying the data with the `PUBLIC` role.

In [ ]:
-- preview data as public with a masking policy
use role public;
select
    *
from
    mp_test.test.customer;  

## Create Clone of Database

Lets take advantage of the Snowflake zero-copy clone in order to do some dev work. In this step we are:

* Creating a Clone
* Issuing Grants to the Clone

In [ ]:
use role accountadmin;
create or replace database mp_test_clone
    clone mp_test;

-- Issue grants to clone
use role accountadmin;
grant usage on database mp_test_clone to role public;
grant usage on schema mp_test_clone.test to role public;
grant select on table mp_test_clone.test.customer to role public;

### Did the Mask Clone?

It looks like its working.

In [ ]:
-- Masking policy cloned to new database
use role public;
select
    *
from
    mp_test_clone.test.customer;

### Let's Confirm the Policy Exists in the Clone

And finally, we can see the masking policy has in fact been cloned with the database.

In [ ]:
-- show policies
use role accountadmin;
show masking policies in account;

## Tear Down

Clean up the objects we created in this demonstration.

In [ ]:
use role accountadmin;
drop database if exists mp_test;
drop database if exists mp_test_clone;
drop warehouse if exists demo_wh;

# Thank you!